In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

app = Flask(__name__)

# Load Hugging Face GPT model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Global sales data variable
sales_data = None

def analyze_sales_data(query):
    """Generate insights using the GPT model."""
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model.generate(inputs.input_ids, max_length=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def load_sales_data():
    """Loads the sales performance data from 'sales_performance_data.csv'."""
    global sales_data
    try:
        sales_data = pd.read_csv('sales_performance_data.csv')
        print("Sales data loaded successfully.")
    except Exception as e:
        print(f"Error loading sales data: {e}")

# Call this function at the start of the application
load_sales_data()

# Endpoint to upload data (optional if you want to allow re-uploading)
@app.route('/api/upload_data', methods=['POST'])
def upload_data():
    global sales_data
    file = request.files['file']
    if file.filename.endswith('.csv'):
        sales_data = pd.read_csv(file)
    elif file.filename.endswith('.json'):
        sales_data = pd.read_json(file)
    else:
        return jsonify({'error': 'Unsupported file format'}), 400
    return jsonify({'message': 'Data uploaded successfully'}), 200

# Individual sales representative performance analysis
@app.route('/api/rep_performance', methods=['GET'])
def rep_performance():
    employee_id = request.args.get('rep_id')

    if not employee_id:
        return jsonify({"error": "rep_id is required"}), 400

    if sales_data is None:
        return jsonify({"error": "No sales data available"}), 404

    # Check if 'employee_id' column exists
    if 'employee_id' not in sales_data.columns:
        return jsonify({"error": "'employee_id' column not found in sales data"}), 500

    # Check if the employee_id is in the DataFrame
    if employee_id not in sales_data['employee_id'].astype(str).unique():
        return jsonify({"error": "No data found for employee_id"}), 404

    # Fetch the performance data for the specific employee_id
    performance_data = sales_data[sales_data['employee_id'].astype(str) == employee_id]
    
    # Return the performance data as a JSON response
    return jsonify(performance_data.to_dict(orient='records')), 200

# Team performance analysis
@app.route('/api/team_performance', methods=['GET'])
def team_performance():
    if sales_data is None:
        return jsonify({'error': 'No sales data available'}), 404

    query = f"Analyze the overall performance of the sales team based on the following data: {sales_data.to_json()}."
    feedback = analyze_sales_data(query)
    return jsonify({'team_feedback': feedback})

# Sales performance trends and forecasting
@app.route('/api/performance_trends', methods=['GET'])
def performance_trends():
    time_period = request.args.get('time_period')
    
    # Check if time_period is valid
    if time_period not in ['monthly', 'quarterly']:
        return jsonify({"error": "Invalid time_period. Use 'monthly' or 'quarterly'."}), 400
    
    if sales_data is None:
        return jsonify({'error': 'No sales data available'}), 404

    query = f"Analyze sales trends and provide forecasts based on {time_period} data: {sales_data.to_json()}."
    feedback = analyze_sales_data(query)
    return jsonify({'performance_trends': feedback})

if __name__ == '__main__':
    app.run(port=4400)


Sales data loaded successfully.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4400
Press CTRL+C to quit
[2024-10-18 19:57:09,020] ERROR in app: Exception on /api/team_performance [GET]
Traceback (most recent call last):
  File "C:\Users\fatima\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fatima\anaconda3\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fatima\anaconda3\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fatima\anaconda3\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fa

In [ ]:
from flask import Flask, jsonify, request
import pandas as pd
from transformers import pipeline
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


# Initialize Flask app
app = Flask(__name__)

data = pd.read_csv("sales_performance_data.csv") 

# Initialize Hugging Face GPT 
model = pipeline('text-generation', model='gpt2')  


# 1. Individual Sales Representative Performance Analysis

@app.route('/api/rep_performance', methods=['GET'])
def rep_performance():
    rep_id = request.args.get('rep_id')
    
    # Find the sales representative data
    rep_data = data[data['employee_id'] == int(rep_id)]
    if rep_data.empty:
        return jsonify({"error": "Representative not found"}), 404
        
    employee_name = rep_data['employee_name'].iloc[0]  # Get the name of the representative 
    # Perform analysis based on the data
    total_leads = int(rep_data['lead_taken'].sum()) 
    total_revenue = float(rep_data['revenue_confirmed'].sum())  
    
    # Generate performance feedback using GPT 
    feedback_input = f"{employee_name} has handled {total_leads} leads and generated ${total_revenue} in confirmed revenue. Provide feedback on this performance."
    feedback = model(feedback_input, max_length=100, num_return_sequences=1, truncation=True)[0]['generated_text']
    
    return jsonify({
        "rep_id": rep_id,
        "employee_name": employee_name,
        "total_leads": total_leads,
        "total_revenue": total_revenue,
        "feedback": feedback
    })

# 2. Overall Sales Team Performance Summary

@app.route('/api/team_performance', methods=['GET'])
def team_performance():
    # Calculate overall team statistics
    total_leads = int(data['lead_taken'].sum())  
    total_revenue = float(data['revenue_confirmed'].sum()) 
    total_tours = int(data['tours_booked'].sum()) 
    
    # Generate feedback on the team's overall performance
    feedback_input = f"The team has collectively taken {total_leads} leads, generated ${total_revenue} in revenue, and booked {total_tours} tours. Provide a summary of the team's performance."
    team_feedback = model(feedback_input, max_length=100, num_return_sequences=1, truncation=True)[0]['generated_text']
    
    return jsonify({
        "total_leads": total_leads,
        "total_revenue": total_revenue,
        "total_tours": total_tours,
        "feedback": team_feedback
    })

# 3. Sales Performance Trends and Forecasting

@app.route('/api/performance_trends', methods=['GET'])
def performance_trends():
    time_period = request.args.get('time_period')
    data['dated'] = pd.to_datetime(data['dated'], errors='coerce') # Ensure 'dated' column is datetime
    
    # Analyze sales performance over the time period (e.g., monthly, quarterly)
    if time_period == 'monthly':
        period_data = data.groupby(data['dated'].dt.to_period('M')).sum(numeric_only=True)
    elif time_period == 'quarterly':
        period_data = data.groupby(data['dated'].dt.to_period('Q')).sum(numeric_only=True)
    else:
        return jsonify({"error": "Invalid time period. Use 'monthly' or 'quarterly'."}), 400

    # Generate trends and forecast feedback
    leads_trend = period_data['lead_taken'].mean()
    revenue_trend = period_data['revenue_confirmed'].mean()
    forecast_input = f"Based on the {time_period} data, the average leads are {leads_trend} and the average revenue is $ {revenue_trend}. Forecast the future performance."
    forecast = model(forecast_input, max_length=100, num_return_sequences=1)[0]['generated_text']

    return jsonify({
        "time_period": time_period,
        "average_leads": leads_trend,
        "average_revenue": revenue_trend,
        "forecast": forecast
    })
    


if __name__ == '__main__':
    app.run(port=3500)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3500
Press CTRL+C to quit
